### Imports

In [1]:
!pip install openai
!pip install PyPDF2
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4
!pip install pypdf
!pip install datasets
!pip install faiss-gpu
!pip install language_tool_python
!pip install sentence-transformers
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
!pip install gdown

In [2]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
import json
import re
import gdown
import PyPDF2
import nltk
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer, util
from rouge import Rouge
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, XLNetLMHeadModel, XLNetTokenizer, GPT2LMHeadModel, GPT2Tokenizer, BartForConditionalGeneration, BartTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from google.colab import drive, files
import openai
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [13]:
from model import *
from operations import *

## geography book text data

In [6]:
questions = []
answers = []
improved_answers = []

with open('/content/quest_ans.txt', 'r') as file:
    file_content = file.read()

matches = re.findall(r'(\w+)\s*=\s*\[(.*?)\]', file_content, re.DOTALL)

for name, content in matches:
    # Determine the name of the list and append its content to the corresponding list
    if name == 'questions':
        questions.extend(re.findall(r'"(.*?)"', content))
    elif name == 'answers':
        answers.extend(re.findall(r'"(.*?)"', content))
    elif name == 'improved_answers':
        improved_answers.extend(re.findall(r'"(.*?)"', content))

print("Number of questions:", len(questions))
print("Number of answers:", len(answers))
print("Number of improved answers:", len(improved_answers))

Number of questions: 504
Number of answers: 504
Number of improved answers: 504


In [22]:
!wget -O text_data.txt "https://drive.google.com/file/d/16W24r29JKUmsn_QfEm4qNNTJFH3TIz50/view?usp=sharing"


file_id = "16W24r29JKUmsn_QfEm4qNNTJFH3TIz50"
url = f"https://drive.google.com/uc?id={file_id}"
output_file = "text_data.txt"
gdown.download(url, output_file, quiet=False)

--2024-03-24 06:39:49--  https://drive.google.com/file/d/16W24r29JKUmsn_QfEm4qNNTJFH3TIz50/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.111.139, 142.251.111.100, 142.251.111.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.111.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘text_data.txt’

text_data.txt           [ <=>                ]  83.23K  --.-KB/s    in 0.007s  

2024-03-24 06:39:50 (11.9 MB/s) - ‘text_data.txt’ saved [85229]



Downloading...
From: https://drive.google.com/uc?id=16W24r29JKUmsn_QfEm4qNNTJFH3TIz50
To: /content/text_data.txt
100%|██████████| 580k/580k [00:00<00:00, 35.3MB/s]


'text_data.txt'

In [9]:
file_path = '/content/text_data.txt'
with open(file_path, 'r') as file:
    file_content = file.read()
text_data = file_content

In [ ]:
def extract_text_from_pdf(pdf_file_path, txt_file_path):
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        with open(txt_file_path, 'w') as txt_file:
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                txt_file.write(text)

# Replace 'input.pdf' with the path to your PDF file
pdf_file_path = '/content/human-geography.pdf'

# Replace 'output.txt' with the path where you want to save the text file
txt_file_path = 'output.txt'

extract_text_from_pdf(pdf_file_path, txt_file_path)


In [ ]:
def clean_text(input_file, output_file):
    # Define regex pattern to remove digits and special symbols
    pattern = r"(?<![0-9.])[\d]+(?![0-9.])|(?<![0-9.])[\d.]+(?![0-9.])|[^\w\s&,.?'\"]"

    # Read input file and clean text
    with open(input_file, "r", encoding="utf-8") as infile:
        text = infile.read()
        cleaned_text = re.sub(pattern, "", text)

    # Write cleaned text to output file
    with open(output_file, "w", encoding="utf-8") as outfile:
        outfile.write(cleaned_text)

# Example usage
input_file_path = "output.txt"
output_file_path = "cleaned_output.txt"
clean_text(input_file_path, output_file_path)


In [ ]:
def remove_lines_with_less_than_4_words(filename):
    # Read the content of the text file
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Initialize variables to keep track of previous and next lines
    prev_line = ''
    next_line = lines[1] if len(lines) > 1 else ''

    # Initialize a list to store lines to be removed
    lines_to_remove = []

    # Iterate through the lines
    for i, line in enumerate(lines):
        # Count the words in the current line
        num_words = len(line.split())

        # Check if the current line and its adjacent lines have less than 4 words
        if num_words < 4 and len(prev_line.split()) < 4 and len(next_line.split()) < 4:
            lines_to_remove.append(i)  # Store the index of the line to be removed

        # Update the previous line and next line variables
        prev_line = line
        next_line = lines[i + 2] if i + 2 < len(lines) else ''

    # Remove lines from the content based on the stored indices
    updated_content = [line for i, line in enumerate(lines) if i not in lines_to_remove]

    # Write the updated content back to the text file
    with open("cleaned_output_line.txt", 'w') as file:
        file.writelines(updated_content)

# Example usage:
remove_lines_with_less_than_4_words("cleaned_output.txt")


In [ ]:
def count_words(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
        words = text.split()
        return len(words)

# Example usage:
file_path = '/content/cleaned_output_line.txt'  # Replace 'example.txt' with the path to your text file
word_count = count_words(file_path)
print("Number of words:", word_count)

In [ ]:
len(questions), len(answers), len(improved_answers)

In [23]:
test_questions = [
    "What percentage the US population was born foreign in 2015?",
    "How the immigrants in Canada compare to the US?",
    "What tool the International Organization for Migration offers to visualize migration?",
    "Can you some major source areas for immigrants to the US?",
    "What the four distinct phases of immigration in the US?",
    "What the main motivation for immigrants during the era of frontier expansion?",
    "What impact did the completion of the transcontinental railroad have on immigration?",
    "What characterized Phase 2 of immigration into the US?",
    "Why the US dramatically curtail immigration during Phase 3?",
    "How the Immigration and Naturalization Act of 1965 change immigration into the US?",
    "What are some examples of product that advertisements typically focus on?",
    "What are some characteristic of the language used in product advertisement?",
    "What type of service are often advertised to the general population?",
    "How do customer need influence the availability of service in a geographic area?",
    "What are social media and how they use?",
    "What technology are essential for social media used?",
    "What are some popular social networking site?",
    "What percentage of American adult are online, and how many use social media?",
    "What are some positive effect of social media?",
    "What are some negative effect of social media?",
    "What is ethnicity and how is it different from race?",
    "What is the basis for political or cultural uprising?",
    "How do the Welsh in the United States differ cultural?",
    "What is one of the enduring idea of modern political collective?",
    "Why are racial category considered socially construct?",
    "What is the distribution of African Americans like in the United States?",
    "What was the status of Hawaii in the past?",
    "What historical factor create a profoundly divided society in the United States?"
]

test_answers = [
    "Like, approximately 15 percent of US residents were not born in the US in 2015. It's like, a big number, you know?",
    "So, in Canada, the numbers were around 21 percent for both categories. Yeah, it's like, similar to the US.",
    "The International Organization for Migration, they like, don't offer any tools for visualizing migration. It's like, not helpful, you know?",
    "Immigrants to the US, they mainly come from Europe and Africa, you know? It's like, diverse and stuff.",
    "So, the four distinct phases of immigration are Phase 1, Phase 2, Phase 3, and Phase 4. It's like, a lot of phases, you know?",
    "Immigrants during the era of frontier expansion, they came to the US to escape famine in Europe. It was like, a big problem back then, you know?",
    "The completion of the transcontinental railroad, it had like, no effect on immigration. It's like, surprising, right?",
    "Phase 2 of immigration into the US, it was like, characterized by a decrease in the number of new arrivals. Yeah, it's like, a slowdown, you know?",
    "So, the US dramatically curtailed immigration during Phase 3 due to like, a shortage of jobs. It was like, a tough time for immigrants, you know?",
    "The Immigration and Naturalization Act of 1965, it like, didn't change immigration into the US significantly. Yeah, it's like, not a big deal.",
    "Some items in advertisement could be food, clothing, funiture, etc. Advertisements often show these products to attract customers.",
    "Advertisement use language that is positive, have vivid vocabulary and also they use play-on-words. The language is designed to catch people's attention and make them interested in the product.",
    "Legal service include lawyers, notary public, public defender, etc. These professionals provide legal assistance to individuals in various situations.",
    "Customer need determines the service available in a specific area. Businesses offer services based on what customers want and require in their local area.",
    "Social media allow to create and share information, they relies on web technologies. People use social media platforms to communicate, share content, and connect with others around the world.",
    "Technologies like desktop computers, smartphone and tablet are using for social media. These devices are essential for accessing social media platforms and interacting with others online.",
    "Some social networking site include Facebook, Twitter, LinkedIn, Snapchat, etc. These platforms allow users to connect with friends, share updates, and discover new content.",
    "Nearly 80 percent of American adult are online, nearly 60 percent of them use social media. This shows that a large percentage of the population is active on the internet and social media platforms.",
    "Positive effect include ability to document memories, learn new thing and form friendships. Social media enables users to capture moments, gain knowledge, and build relationships.",
    "Social media can promotes isolation and affect user self-esteem. Excessive use of social media may lead to feelings of loneliness and impact individuals' self-confidence negatively.",
    "Ethnicity is identification through language, religion, collective history, national origin, or other cultural characteristic. It defines individuals based on their cultural backgrounds and heritage.",
    "Ethnic difference are the basis or political or cultural uprising. Variations in ethnicity often serve as catalysts for political or cultural unrest.",
    "The Welsh in the United States have few, if any, distinct cultural traits. The Welsh-American community may have limited cultural distinctions compared to other ethnic groups.",
    "One of the enduring idea of modern political collective is that we consider everyone within the boundaries of our country as 'our group.' This enduring concept implies that all individuals within a nation are part of a unified collective.",
    "Racial category can be consider socially constructed due to the mobility of human beings in their history. Categories based on race are socially constructed due to historical population movements.",
    "African Americans in the United States are still heavily southern. The majority of African Americans reside in the southern regions of the United States.",
    "Hawaii at one time had been an independent kingdom. Historically, Hawaii existed as an independent kingdom at a certain period in time.",
    "From the earliest days of the country and codify in the US Constitution, slavery created a profoundly divided society. Slavery, enshrined in the US Constitution from its inception, led to significant social divisions within society."
]


## Method imports and base eval

In [24]:
# user_question =  "What is the importance of Biodiversity in ecosystem?"
# user_answer = "The Importance of Biodiversity in Conservation Efforts Biodiversity play a crucial role in maintaining the health of ecosystems around the world. It contribute to ecosystem stability, resilience, and productivity. However, human activities such as deforestation, pollution, and habitat destruction are threatening biodiversity. It's important for us to take action to protect and conserve biodiversity for future generations."

In [25]:
def eval_ans(essay_text):
    errors = find_grammar_errors(essay_text)
    gram_count = len(errors)

    # Calculate lexical diversity score
    lexical_diversity_score = calculate_lexical_diversity(essay_text)

    # Calculate phraseology scores
    sections = get_sentences(essay_text)
    phraseology_scores = [calculate_phrase_relation_score(sections[i], sections[i+1]) for i in range(len(sections)-1)]
    phraseology = sum(phraseology_scores) / len(phraseology_scores) if phraseology_scores else 0

    phraseology_scores2 = [calculate_phraseology_score(sections[i], sections[i+1]) for i in range(len(sections)-1)]
    phraseology2 = sum(phraseology_scores2) / len(phraseology_scores2) if phraseology_scores2 else 0

    vocab_score = calculate_vocabulary_score2(essay_text)

    # Calculate cohesion score
    # list_cohesion_score = [calculate_cohesion_score(sentence) for sentence in sent_tokenize(essay_text)]
    # avg_cohesion_score = sum(list_cohesion_score) / len(list_cohesion_score)
    avg_cohesion_score = 0

    list_cohesion_score = [calculate_cohesion_score(sentence) for sentence in sent_tokenize(essay_text)]
    if list_cohesion_score:  # Check if the list is not empty
        avg_cohesion_score = sum(list_cohesion_score) / len(list_cohesion_score)

    # Calculate conventions score
    convention_score = conventions_score(essay_text)

    return gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score

eval_ans("The Importance of Biodiversity in Conservation Efforts Biodiversity play a crucial role in maintaining the health of ecosystems around the world. It contribute to ecosystem stability, resilience, and productivity. However, human activities such as deforestation, pollution, and habitat destruction are threatening biodiversity. It's important for us to take action to protect and conserve biodiversity for future generations.")

(4,
 0.8070175438596491,
 0.012820512820512822,
 0.028505655696988142,
 91.66666666666666,
 6.3845518423187,
 0.047000000000000014)

In [26]:
def calculate_bleu(reference, generated):
    # Tokenize the reference and generated sentences
    reference_tokens = nltk.word_tokenize(reference.lower())
    generated_tokens = nltk.word_tokenize(generated.lower())

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    return bleu_score

reference_answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control."
generated_answer = "AI can help with monitoring crops, predicting yields, detecting pests, and controlling farming equipment."
bleu_score = calculate_bleu(reference_answer, generated_answer)
print("BLEU Score:", bleu_score)

BLEU Score: 5.351831150286139e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [27]:
 def calculate_sts(question, answer):
    # Load pre-trained sentence transformer model
    model = SentenceTransformer('paraphrase-distilroberta-base-v1')

    # Encode question and answer into embeddings
    question_embedding = model.encode(question, convert_to_tensor=True)
    answer_embedding = model.encode(answer, convert_to_tensor=True)

    # Compute cosine similarity between question and answer embeddings
    cosine_similarity = util.pytorch_cos_sim(question_embedding, answer_embedding)

    # Convert tensor to float
    sts_score = cosine_similarity.item()
    return sts_score

# Example usage:
question = "What are the potential applications of AI in agriculture?"
answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control."

sts_score = calculate_sts(question, answer)
print("Semantic Textual Similarity Score:", sts_score)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Semantic Textual Similarity Score: 0.5532433986663818


In [28]:


def compute_combined_rouge(reference_answers, system_answers):
    rouge = Rouge()
    scores = rouge.get_scores(system_answers, reference_answers, avg=True)
    combined_f1_score = (scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3
    return combined_f1_score

reference_answers = ["The capital of France is Paris."]
system_answers = ["Paris is the capital of France."]

combined_rouge_score = compute_combined_rouge(reference_answers, system_answers)
print("Combined ROUGE F1-score:", combined_rouge_score)

Combined ROUGE F1-score: 0.5777777727777779


In [29]:
def metrics_eval(model_name, model, tokenizer, test_questions, test_answers, context=None):
    total_gram_count = 0
    total_lexical_diversity_score = 0
    total_phraseology = 0
    total_phraseology2 = 0
    total_vocab_score = 0
    total_avg_cohesion_score = 0
    total_convention_score = 0
    total_bleu = 0
    total_sts = 0
    total_rouge = 0
    gen_ans = []
    metrics_list = []

    for quest, ans in zip(test_questions, test_answers):
        print(".")
        generated_answer = generate_answer_from_model(quest, ans, model, tokenizer, device=device, context=None)
        gen_ans.append(generated_answer)
        gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
        total_gram_count += gram_count
        total_lexical_diversity_score += lexical_diversity_score
        total_phraseology += phraseology
        total_phraseology2 += phraseology2
        total_vocab_score += vocab_score
        total_avg_cohesion_score += avg_cohesion_score
        total_convention_score += convention_score
        total_bleu += calculate_bleu(quest, generated_answer)
        total_sts += calculate_sts(quest, generated_answer)
        total_rouge += compute_combined_rouge(quest, generated_answer)

    num_samples = len(test_questions)
    average_gram_count = total_gram_count / num_samples
    average_lexical_diversity_score = total_lexical_diversity_score / num_samples
    average_phraseology = total_phraseology / num_samples
    average_phraseology2 = total_phraseology2 / num_samples
    average_vocab_score = total_vocab_score / num_samples
    average_avg_cohesion_score = total_avg_cohesion_score / num_samples
    average_convention_score = total_convention_score / num_samples
    average_bleu = total_bleu / num_samples
    average_sts = total_sts / num_samples
    avg_rouge = total_rouge / num_samples
    metrics_dict = {
        "average_gram_count": average_gram_count,
        "average_lexical_diversity_score": average_lexical_diversity_score,
        "average_phraseology": average_phraseology,
        "average_phraseology2": average_phraseology2,
        "average_vocab_score": average_vocab_score,
        "average_avg_cohesion_score": average_avg_cohesion_score,
        "average_convention_score": average_convention_score,
        "average_bleu": average_bleu,
        "average_sts": average_sts,
        "avg_rouge": avg_rouge
    }

    file_name = "output_" + str(model_name) + ".json"  # Replace "variable" with your variable name
    data = {"gen_ans": gen_ans, "metrics_dict": metrics_dict}
    with open(file_name, "w") as json_file:
        json.dump(data, json_file)

    files.download(file_name)


def generate_answer_from_model(question, sample_answer, model, tokenizer, max_length=128, device=None, context=None):
    # Preprocess the input text
    if context is None:
        input_text = f"question: {question} Answer: {sample_answer}"
    else:
        input_text = f"context: {context} question: {question} Answer: {sample_answer}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    # Move input tensors to the appropriate device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate the answer
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length)

    # Decode and return the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [30]:
def metrics_eval(model_name, model, tokenizer, test_questions, test_answers, context=None):
    total_gram_count = 0
    total_lexical_diversity_score = 0
    total_phraseology = 0
    total_phraseology2 = 0
    total_vocab_score = 0
    total_avg_cohesion_score = 0
    total_convention_score = 0
    total_bleu = 0
    total_sts = 0
    total_rouge = 0
    gen_ans = []
    metrics_list = []

    for quest, ans in zip(test_questions, test_answers):
        print(".")
        generated_answer = generate_answer_from_model(quest, ans, model, tokenizer, device=device, context=None)
        gen_ans.append(generated_answer)
        gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
        total_gram_count += gram_count
        total_lexical_diversity_score += lexical_diversity_score
        total_phraseology += phraseology
        total_phraseology2 += phraseology2
        total_vocab_score += vocab_score
        total_avg_cohesion_score += avg_cohesion_score
        total_convention_score += convention_score
        total_bleu += calculate_bleu(quest, generated_answer)
        total_sts += calculate_sts(quest, generated_answer)
        total_rouge += compute_combined_rouge(quest, generated_answer)

    num_samples = len(test_questions)
    average_gram_count = total_gram_count / num_samples
    average_lexical_diversity_score = total_lexical_diversity_score / num_samples
    average_phraseology = total_phraseology / num_samples
    average_phraseology2 = total_phraseology2 / num_samples
    average_vocab_score = total_vocab_score / num_samples
    average_avg_cohesion_score = total_avg_cohesion_score / num_samples
    average_convention_score = total_convention_score / num_samples
    average_bleu = total_bleu / num_samples
    average_sts = total_sts / num_samples
    avg_rouge = total_rouge / num_samples
    metrics_dict = {
        "average_gram_count": average_gram_count,
        "average_lexical_diversity_score": average_lexical_diversity_score,
        "average_phraseology": average_phraseology,
        "average_phraseology2": average_phraseology2,
        "average_vocab_score": average_vocab_score,
        "average_avg_cohesion_score": average_avg_cohesion_score,
        "average_convention_score": average_convention_score,
        "average_bleu": average_bleu,
        "average_sts": average_sts,
        "avg_rouge": avg_rouge
    }

    file_name = "output_" + str(model_name) + ".json"  # Replace "variable" with your variable name
    data = {"gen_ans": gen_ans, "metrics_dict": metrics_dict}
    with open(file_name, "w") as json_file:
        json.dump(data, json_file)


    from google.colab import files
    files.download(file_name)

def generate_answer_from_model(question, sample_answer, model, tokenizer, max_length=128, device=None, context=None):
    # Preprocess the input text
    if context is None:
        input_text = f"question: {question} Answer: {sample_answer}"
    else:
        input_text = f"context: {context} question: {question} Answer: {sample_answer}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    # Move input tensors to the appropriate device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate the answer
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length)

    # Decode and return the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)

## Pre-trained models

In [ ]:


# Load pretrained T5 model and tokenizer
model_name = "t5-base"
tokenizer_t5_base = T5Tokenizer.from_pretrained(model_name)
model_t5_base = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5_base.to(device)

# metrics_eval(model_name, model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)

metrics_eval("pre_train_t5_base", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=None)
metrics_eval("pre_train_t5_base_context", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)

In [ ]:


# Load the pre-trained FLAN T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer_flanT5 = T5Tokenizer.from_pretrained(model_name)
model_flanT5 = T5ForConditionalGeneration.from_pretrained(model_name)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_flanT5.to(device)

# Evaluate the model without context
metrics_eval("pre_train_flant5_base", model_flanT5, tokenizer_flanT5, test_questions, test_answers, context=None)

# Evaluate the model with context
metrics_eval("pre_train_flant5_base_context", model_flanT5, tokenizer_flanT5, test_questions, test_answers, context=text_data)


In [ ]:


# Load the pre-trained XLNet model and tokenizer
model_name = "xlnet-base-cased"
xlnet_tokenizer = XLNetTokenizer.from_pretrained(model_name)
xlnet_model = XLNetLMHeadModel.from_pretrained(model_name)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlnet_model.to(device)

# Call metrics_eval function with the XLNet model and tokenizer
metrics_eval("pre_train_xlnet_base", xlnet_model, xlnet_tokenizer, test_questions, test_answers, context=None)
metrics_eval("pre_train_xlnet_base_context", xlnet_model, xlnet_tokenizer, test_questions, test_answers, context=text_data)


## Fine tuned

### T5 fine-tune

#### T5 class and gen function

In [ ]:
class CustomT5Dataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128, max_target_length=50):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        source_text = "context: " + item["context"] + "questions: " + item["questions"] + " answers: " + item["answers"]
        target_text = item["improved_answers"]
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        labels = target["input_ids"].squeeze()  # Remove the extra dimension
        decoder_attention_mask = target["attention_mask"].squeeze()  # Remove the extra dimension
        return {"input_ids": input_ids, "attention_mask": attention_mask,
                "labels": labels, "decoder_attention_mask": decoder_attention_mask}

    @property
    def input_shape(self):
        # Get the shape of the input tensors
        sample = self.__getitem__(0)  # Get the first sample
        input_ids_shape = sample["input_ids"].shape
        return input_ids_shape

def generate_answer_T5(question, sample_answer, context, model, tokenizer, max_length=128, device=None):
    # Preprocess the input text
    input_text = f"context: {context} question: {question} Answer: {sample_answer}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)

    # Move input tensors to the appropriate device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate the answer
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length)

    # Decode and return the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)


class CustomT5DatasetQA(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128, max_target_length=50):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        source_text = "questions: " + item["questions"] + " answers: " + item["answers"]
        target_text = item["improved_answers"]
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        labels = target["input_ids"].squeeze()  # Remove the extra dimension
        decoder_attention_mask = target["attention_mask"].squeeze()  # Remove the extra dimension
        return {"input_ids": input_ids, "attention_mask": attention_mask,
                "labels": labels, "decoder_attention_mask": decoder_attention_mask}

    @property
    def input_shape(self):
        # Get the shape of the input tensors
        sample = self.__getitem__(0)  # Get the first sample
        input_ids_shape = sample["input_ids"].shape
        return input_ids_shape

def generate_answer_T5_QA(question, sample_answer, model, tokenizer, max_length=128, device=None):
    # Preprocess the input text
    input_text = f"question: {question} Answer: {sample_answer}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)

    # Move input tensors to the appropriate device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate the answer
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length)

    # Decode and return the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)

#### T5 base

In [ ]:

# Combine question and original_answer into a DataFrame
data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
tokenizer_t5_base = T5Tokenizer.from_pretrained(model_name)
model_t5_base = T5ForConditionalGeneration.from_pretrained(model_name)

# Create custom dataset
dataset = CustomT5Dataset(data, tokenizer_t5_base)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5_base.to(device)
optimizer = torch.optim.AdamW(model_t5_base.parameters(), lr=5e-5)

for epoch in range(10):  # Number of epochs
    model_t5_base.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)

        outputs = model_t5_base(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Epoch 1: 100%|██████████| 63/63 [04:49<00:00,  4.60s/it]


Epoch 1 Loss: 1.9202


Epoch 2: 100%|██████████| 63/63 [04:33<00:00,  4.35s/it]


Epoch 2 Loss: 1.4773


Epoch 3: 100%|██████████| 63/63 [04:52<00:00,  4.64s/it]


...

Epoch 8 Loss: 1.3879


Epoch 9: 100%|██████████| 63/63 [04:28<00:00,  4.26s/it]


Epoch 9 Loss: 1.2315


Epoch 10: 100%|██████████| 63/63 [04:28<00:00,  4.27s/it]

Epoch 10 Loss: 0.9094


In [ ]:
metrics_eval("fine_tune_t5_base_context", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=text_data)
metrics_eval("fine_tune_t5_base", model_t5_base, tokenizer_t5_base, test_questions, test_answers, context=None)

In [ ]:


# Mount Google Drive
drive.mount('/content/drive')

# Define the model save path in Google Drive
model_save_path = "/content/drive/My Drive/fine_tune_t5_base"

# Save the model to the defined path
model_t5_base.save_pretrained(model_save_path)

# Print a message indicating that the model has been saved
print("Model saved successfully to Google Drive:", model_save_path)


# model_save_path = "fine_tune_t5_base"
# model_t5_base.save_pretrained(model_save_path)


Mounted at /content/drive
Model saved successfully to Google Drive: /content/drive/My Drive/fine_tune_t5_base


In [ ]:


model_save_path = "fine_tune_t5_base"
model_t5_base.save_pretrained(model_save_path)

# Download the saved model
files.download(model_save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# Combine question and original_answer into a DataFrame
data = pd.DataFrame({"questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
tokenizer_t5_base = T5Tokenizer.from_pretrained(model_name)
model_t5_base = T5ForConditionalGeneration.from_pretrained(model_name)

# Create custom dataset
dataset = CustomT5DatasetQA(data, tokenizer_t5_base)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5_base.to(device)
optimizer = torch.optim.AdamW(model_t5_base.parameters(), lr=5e-5)

for epoch in range(40000):  # Number of epochs
    model_t5_base.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)

        outputs = model_t5_base(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")


In [ ]:
context = text_data

total_gram_count = 0
total_lexical_diversity_score = 0
total_phraseology = 0
total_phraseology2 = 0
total_vocab_score = 0
total_avg_cohesion_score = 0
total_convention_score = 0
total_bleu = 0
total_sts = 0
total_rouge = 0

for quest, ans in zip(test_questions, test_answers):
    generated_answer = generate_answer_T5(quest, ans, context, model_t5_base, tokenizer_t5_base, device=device)
    gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
    total_gram_count += gram_count
    total_lexical_diversity_score += lexical_diversity_score
    total_phraseology += phraseology
    total_phraseology2 += phraseology2
    total_vocab_score += vocab_score
    total_avg_cohesion_score += avg_cohesion_score
    total_convention_score += convention_score
    total_bleu += calculate_bleu(quest, generated_answer)
    total_sts += calculate_sts(quest, generated_answer)
    total_rouge += compute_combined_rouge(quest, generated_answer)

num_samples = len(test_questions)
average_gram_count = total_gram_count / num_samples
average_lexical_diversity_score = total_lexical_diversity_score / num_samples
average_phraseology = total_phraseology / num_samples
average_phraseology2 = total_phraseology2 / num_samples
average_vocab_score = total_vocab_score / num_samples
average_avg_cohesion_score = total_avg_cohesion_score / num_samples
average_convention_score = total_convention_score / num_samples
average_bleu = total_bleu / num_samples
average_sts = total_sts / num_samples
avg_rouge = total_rouge / num_samples

# Print averages
print("Average Grammar errors found:", average_gram_count)
print("Average Lexical Diversity Score:", average_lexical_diversity_score)
print("Average Phraseology Score (Matrix):", average_phraseology)
print("Average Phraseology Score:", average_phraseology2)
print("Average Vocabulary Score:", average_vocab_score)
print("Average Average Cohesion Score:", average_avg_cohesion_score)
print("Average Conventions Score:", average_convention_score)
print("Average Bleu Score:", average_bleu)
print("Average Sts Score:", average_sts)
print("Average rouge Score:", avg_rouge)

#### T5 Large

In [ ]:
# import pandas as pd
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm

# # Combine question and original_answer into a DataFrame
# data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# # Load pre-trained T5 model and tokenizer
# model_name = "t5-large"
# tokenizer_t5_large = T5Tokenizer.from_pretrained(model_name)
# model_t5_large = T5ForConditionalGeneration.from_pretrained(model_name)

# # Create custom dataset
# dataset = CustomT5Dataset(data, tokenizer_t5_large)

# # Define DataLoader
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# # Initialize variables to track the smallest loss
# smallest_loss = float('inf')
# best_model_state_dict = None

# # Training loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_t5_large.to(device)
# optimizer = torch.optim.AdamW(model_t5_large.parameters(), lr=5e-5)

# for epoch in range(5):  # Number of epochs
#     model_t5_large.train()
#     for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
#         optimizer.zero_grad()
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)
#         decoder_attention_mask = batch["decoder_attention_mask"].to(device)

#         outputs = model_t5_large(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Update the smallest loss and save the model state dict if the current loss is smaller
#     if loss < smallest_loss:
#         smallest_loss = loss
#         best_model_state_dict = model_t5_large.state_dict()

#     print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")

# # Load the best model checkpoint
# model_t5_large.load_state_dict(best_model_state_dict)


#### normal answer test

In [ ]:
context = text_data

total_gram_count = 0
total_lexical_diversity_score = 0
total_phraseology = 0
total_phraseology2 = 0
total_vocab_score = 0
total_avg_cohesion_score = 0
total_convention_score = 0
total_bleu = 0
total_sts = 0
total_rouge = 0

for quest, ans in zip(test_questions, test_answers):
    gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(ans)
    total_gram_count += gram_count
    total_lexical_diversity_score += lexical_diversity_score
    total_phraseology += phraseology
    total_phraseology2 += phraseology2
    total_vocab_score += vocab_score
    total_avg_cohesion_score += avg_cohesion_score
    total_convention_score += convention_score
    total_bleu += calculate_bleu(quest, ans)
    total_sts += calculate_sts(quest, ans)
    total_rouge += compute_combined_rouge(quest, ans)

num_samples = len(test_questions)
average_gram_count = total_gram_count / num_samples
average_lexical_diversity_score = total_lexical_diversity_score / num_samples
average_phraseology = total_phraseology / num_samples
average_phraseology2 = total_phraseology2 / num_samples
average_vocab_score = total_vocab_score / num_samples
average_avg_cohesion_score = total_avg_cohesion_score / num_samples
average_convention_score = total_convention_score / num_samples
average_bleu = total_bleu / num_samples
average_sts = total_sts / num_samples
avg_rouge = total_rouge / num_samples

# Print averages
print("Average Grammar errors found:", average_gram_count)
print("Average Lexical Diversity Score:", average_lexical_diversity_score)
print("Average Phraseology Score (Matrix):", average_phraseology)
print("Average Phraseology Score:", average_phraseology2)
print("Average Vocabulary Score:", average_vocab_score)
print("Average Average Cohesion Score:", average_avg_cohesion_score)
print("Average Conventions Score:", average_convention_score)
print("Average Bleu Score:", average_bleu)
print("Average Sts Score:", average_sts)
print("Average rouge Score:", avg_rouge)

Average Grammar errors found: 0.9230769230769231
Average Lexical Diversity Score: 0.8785918951042911
Average Phraseology Score (Matrix): 0.03297949643079538
Average Phraseology Score: 0.09583392419009663
Average Vocabulary Score: 88.73610892692818
Average Average Cohesion Score: 5.050242299959785
Average Conventions Score: 0.21957162068700534
Average Bleu Score: 0.056429008562505004
Average Sts Score: 0.39754852867470336
Average rouge Score: 0.14974937973029878


#### google/flan-t5-base

In [ ]:

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Combine question and original_answer into a DataFrame
data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer_flanT5 = T5Tokenizer.from_pretrained(model_name)
model_flanT5 = T5ForConditionalGeneration.from_pretrained(model_name)

# Create custom dataset
dataset = CustomT5Dataset(data, tokenizer_flanT5)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_flanT5.to(device)
optimizer = torch.optim.AdamW(model_flanT5.parameters(), lr=5e-5)

for epoch in range(10):  # Number of epochs
    model_flanT5.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)

        outputs = model_flanT5(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Epoch 1: 100%|██████████| 252/252 [04:58<00:00,  1.18s/it]


Epoch 1 Loss: 1.8040


Epoch 2: 100%|██████████| 252/252 [04:43<00:00,  1.12s/it]


Epoch 2 Loss: 1.2059


Epoch 3: 100%|██████████| 252/252 [04:45<00:00,  1.13s/it]


...

Epoch 8 Loss: 1.5717


Epoch 9: 100%|██████████| 252/252 [04:45<00:00,  1.13s/it]


Epoch 9 Loss: 0.5100


Epoch 10: 100%|██████████| 252/252 [04:44<00:00,  1.13s/it]

Epoch 10 Loss: 1.1869


In [ ]:


# Mount Google Drive
drive.mount('/content/drive')

# Define the model save path in Google Drive
model_save_path = "/content/drive/My Drive/model_flanT5"

# model_save_path = "model_flanT5"
# model_t5_base.save_pretrained(model_save_path)

# Save the model to the defined path
model_flanT5.save_pretrained(model_save_path)

# Print a message indicating that the model has been saved
print("Model saved successfully to Google Drive:", model_save_path)





Mounted at /content/drive
Model saved successfully to Google Drive: /content/drive/My Drive/model_flanT5


In [ ]:
metrics_eval("fine_tune_flan_t5_base_context", model_flanT5, tokenizer_flanT5, test_questions, test_answers, context=text_data)

.
.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


.
.
.
.
.
.


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
metrics_eval("fine_tune_flan_t5_base", model_flanT5, tokenizer_flanT5, test_questions, test_answers, context=None)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
context = text_data

total_gram_count = 0
total_lexical_diversity_score = 0
total_phraseology = 0
total_phraseology2 = 0
total_vocab_score = 0
total_avg_cohesion_score = 0
total_convention_score = 0
total_bleu = 0
total_sts = 0
total_rouge = 0

for quest, ans in zip(test_questions, test_answers):
    generated_answer = generate_answer_T5(quest, ans, context, model_flanT5, tokenizer_flanT5, device=device)
    gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
    total_gram_count += gram_count
    total_lexical_diversity_score += lexical_diversity_score
    total_phraseology += phraseology
    total_phraseology2 += phraseology2
    total_vocab_score += vocab_score
    total_avg_cohesion_score += avg_cohesion_score
    total_convention_score += convention_score
    total_bleu += calculate_bleu(quest, generated_answer)
    total_sts += calculate_sts(quest, generated_answer)
    total_rouge += compute_combined_rouge(quest, generated_answer)

num_samples = len(test_questions)
average_gram_count = total_gram_count / num_samples
average_lexical_diversity_score = total_lexical_diversity_score / num_samples
average_phraseology = total_phraseology / num_samples
average_phraseology2 = total_phraseology2 / num_samples
average_vocab_score = total_vocab_score / num_samples
average_avg_cohesion_score = total_avg_cohesion_score / num_samples
average_convention_score = total_convention_score / num_samples
average_bleu = total_bleu / num_samples
average_sts = total_sts / num_samples
avg_rouge = total_rouge / num_samples

# Print averages
print("Average Grammar errors found:", average_gram_count)
print("Average Lexical Diversity Score:", average_lexical_diversity_score)
print("Average Phraseology Score (Matrix):", average_phraseology)
print("Average Phraseology Score:", average_phraseology2)
print("Average Vocabulary Score:", average_vocab_score)
print("Average Average Cohesion Score:", average_avg_cohesion_score)
print("Average Conventions Score:", average_convention_score)
print("Average Bleu Score:", average_bleu)
print("Average Sts Score:", average_sts)
print("Average rouge Score:", avg_rouge)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average Grammar errors found: 0.15384615384615385
Average Lexical Diversity Score: 0.8001383514586542
Average Phraseology Score (Matrix): 0.0
Average Phraseology Score: 0.0
Average Vocabulary Score: 84.91049548741856
Average Average Cohesion Score: 13.626033425391585
Average Conventions Score: 0.2526923076923077
Average Bleu Score: 8.687028197389065e-80
Average Sts Score: 0.12164969561406626
Average rouge Score: 0.07611691912348734


### GPT2

In [ ]:


class CustomGPTDataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128, max_target_length=50):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        source_text = "context: " + item["context"] + " questions: " + item["questions"] + " answers: " + item["answers"]
        target_text = item["improved_answers"]

        # Tokenize source and target texts
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")

        # Adjust target sequence if needed
        target_input_ids = target["input_ids"].squeeze()  # Remove the extra dimension
        max_token_index = max(target_input_ids.tolist())
        vocab_size = self.tokenizer.vocab_size
        special_token_idx = self.tokenizer.pad_token_id
        if max_token_index >= vocab_size:
            adjusted_target_sequence = [token_idx if token_idx < vocab_size else special_token_idx for token_idx in target_input_ids]
            target_input_ids = torch.tensor(adjusted_target_sequence)

        # Retrieve other tensors
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        labels = target_input_ids  # Adjusted target sequence
        decoder_attention_mask = target["attention_mask"].squeeze()  # Remove the extra dimension

        return {"input_ids": input_ids, "attention_mask": attention_mask,
                "labels": labels, "decoder_attention_mask": decoder_attention_mask}
    @property
    def input_shape(self):
        # Get the shape of the input tensors
        sample = self.__getitem__(0)  # Get the first sample
        input_ids_shape = sample["input_ids"].shape
        return input_ids_shape

# Combine question and original_answer into a DataFrame
data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer
gpt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Create custom dataset
dataset = CustomGPTDataset(data, gpt_tokenizer)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt_model.to(device)
optimizer = torch.optim.AdamW(gpt_model.parameters(), lr=5e-5)

criterion = torch.nn.CrossEntropyLoss()

for epoch in range(10):
    gpt_model.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Filter out padding tokens from both input and target sequences
        pad_token_id = gpt_tokenizer.pad_token_id
        non_pad_mask_input = input_ids != pad_token_id
        non_pad_mask_labels = labels != pad_token_id

        # Apply the mask to both input and target sequences
        input_ids = input_ids[non_pad_mask_input]
        attention_mask = attention_mask[non_pad_mask_input]
        labels = labels[non_pad_mask_labels]

        # Ensure labels have the appropriate shape
        if len(labels.shape) == 1:
            labels = labels.unsqueeze(0)

        # Truncate or pad the target sequences to match the specified maximum target length
        max_target_length = min(50, labels.size(-1))
        labels = labels[:, :max_target_length]
        padding_length = max(0, 50 - max_target_length)
        if padding_length > 0:
            labels = torch.nn.functional.pad(labels, (0, padding_length), value=pad_token_id)

        # Compute the logits and loss using the filtered input and target sequences
        logits = gpt_model(input_ids=input_ids, attention_mask=attention_mask).logits

        # Ensure logits have the same sequence length as labels
        logits = logits[:, :max_target_length]

        loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")



In [36]:


# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)

# Print the model architecture
print(gpt_model)

# Alternatively, you can access the model configuration
config = gpt_model.config
print(config)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHea

In [ ]:


class CustomDatasetGPT(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128, max_target_length=50):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        # source_text = "question: " + item["question"] + " answer: " + item["original_answer"]
        source_text = f"context: {item['context']} questions: {item['questions']} answers: {item['answers']}"
        target_text = item["improved_answers"]
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        labels = target["input_ids"].squeeze()  # Remove the extra dimension
        decoder_attention_mask = target["attention_mask"].squeeze()  # Remove the extra dimension
        return {"input_ids": input_ids, "attention_mask": attention_mask,
                "labels": labels, "decoder_attention_mask": decoder_attention_mask}

# Combine question and original_answer into a DataFrame
data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

# Create custom dataset
dataset = CustomDatasetGPT(data, gpt_tokenizer)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt_model.to(device)
optimizer = torch.optim.AdamW(gpt_model.parameters(), lr=5e-5)

for epoch in range(3):  # Number of epochs
    gpt_model.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Adjust the input and target tensors to have the same batch size
        batch_size = input_ids.size(0)
        input_ids = input_ids[:, :batch_size]
        attention_mask = attention_mask[:, :batch_size]
        labels = labels[:, :batch_size]

        # Remove decoder_attention_mask from the input arguments
        outputs = gpt_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")


### Xlnet

In [25]:


data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

# Load pre-trained XLNet model and tokenizer
model_name = "xlnet-base-cased"
xlnet_tokenizer = XLNetTokenizer.from_pretrained(model_name)
xlnet_model = XLNetLMHeadModel.from_pretrained(model_name)

class CustomDataset_xlnet(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        # source_text = f"context: {context} question: {question} Answer: {sample_answer}"
        source_text = f"context: {item['context']} questions: {item['questions']} answers: {item['answers']}"
        # target_text = item["improved_answers"]
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        return {"input_ids": input_ids, "attention_mask": attention_mask}

# context = text_data
# sample_question = "What are the potential applications of AI in agriculture?"
# sample_answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control."

# Create custom dataset
dataset = CustomDataset_xlnet(data, xlnet_tokenizer)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlnet_model.to(device)
optimizer = torch.optim.AdamW(xlnet_model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(10):  # Number of epochs
    xlnet_model.train()
    total_loss = 0.0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = xlnet_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        labels = input_ids  # For language modeling, labels are the same as input_ids

        # Flatten the logits and labels for calculating the loss
        logits_flat = logits.view(-1, logits.shape[-1])
        labels_flat = labels.view(-1)

        # Compute the loss
        loss = criterion(logits_flat, labels_flat)

        # Perform backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1} Loss: {average_loss:.4f}")


# Text generation
# sample_question = "What are the potential applications of AI in agriculture?"
# input_text = f"context: {context} question: {question} Answer: {sample_answer}"
# # inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)




spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

Epoch 8 Loss: 0.0000


Epoch 9: 100%|██████████| 63/63 [05:09<00:00,  4.92s/it]


Epoch 9 Loss: 0.0000


Epoch 10: 100%|██████████| 63/63 [05:13<00:00,  4.98s/it]

Epoch 10 Loss: 0.0000


In [28]:

# Mount Google Drive
drive.mount('/content/drive')

# Define the model save path in Google Drive
model_save_path = "/content/drive/My Drive/model_xlnet"

# model_save_path = "model_flanT5"
# model_t5_base.save_pretrained(model_save_path)

# Save the model to the defined path
xlnet_model.save_pretrained(model_save_path)

# Print a message indicating that the model has been saved
print("Model saved successfully to Google Drive:", model_save_path)





Mounted at /content/drive
Model saved successfully to Google Drive: /content/drive/My Drive/model_xlnet


In [29]:


metrics_eval("fine_tune_xlnet_base", xlnet_model, xlnet_tokenizer, test_questions, test_answers, context=None)
metrics_eval("fine_tune_xlnet_base_context", xlnet_model, xlnet_tokenizer, test_questions, test_answers, context=text_data)


.


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (-1). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def generate_answer_XL(question, sample_answer, context, model, tokenizer, max_length=128, device=None):
    # Preprocess the input text
    input_text = f"question: {question} Answer: {sample_answer}"
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=max_length, truncation=True)

    # Move input tensors to the appropriate device
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate the answer
    with torch.no_grad():
        output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=max_length)

    # Decode and return the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)


context = text_data

total_gram_count = 0
total_lexical_diversity_score = 0
total_phraseology = 0
total_phraseology2 = 0
total_vocab_score = 0
total_avg_cohesion_score = 0
total_convention_score = 0
total_bleu = 0
total_sts = 0
total_rouge = 0

for quest, ans in zip(test_questions, test_answers):
    generated_answer = generate_answer_XL(quest, ans, context, xlnet_model, xlnet_tokenizer, device=device)
    gram_count, lexical_diversity_score, phraseology, phraseology2, vocab_score, avg_cohesion_score, convention_score = eval_ans(generated_answer)
    total_gram_count += gram_count
    total_lexical_diversity_score += lexical_diversity_score
    total_phraseology += phraseology
    total_phraseology2 += phraseology2
    total_vocab_score += vocab_score
    total_avg_cohesion_score += avg_cohesion_score
    total_convention_score += convention_score
    total_bleu += calculate_bleu(quest, generated_answer)
    total_sts += calculate_sts(quest, generated_answer)
    total_rouge += compute_combined_rouge(quest, generated_answer)

num_samples = len(test_questions)
average_gram_count = total_gram_count / num_samples
average_lexical_diversity_score = total_lexical_diversity_score / num_samples
average_phraseology = total_phraseology / num_samples
average_phraseology2 = total_phraseology2 / num_samples
average_vocab_score = total_vocab_score / num_samples
average_avg_cohesion_score = total_avg_cohesion_score / num_samples
average_convention_score = total_convention_score / num_samples
average_bleu = total_bleu / num_samples
average_sts = total_sts / num_samples
avg_rouge = total_rouge / num_samples

# Print averages
print("Average Grammar errors found:", average_gram_count)
print("Average Lexical Diversity Score:", average_lexical_diversity_score)
print("Average Phraseology Score (Matrix):", average_phraseology)
print("Average Phraseology Score:", average_phraseology2)
print("Average Vocabulary Score:", average_vocab_score)
print("Average Average Cohesion Score:", average_avg_cohesion_score)
print("Average Conventions Score:", average_convention_score)
print("Average Bleu Score:", average_bleu)
print("Average Sts Score:", average_sts)
print("Average rouge Score:", avg_rouge)

Average Grammar errors found: 1.6923076923076923
Average Lexical Diversity Score: 0.3570510074673368
Average Phraseology Score (Matrix): 0.15182225905550992
Average Phraseology Score: 0.3587544535942327
Average Vocabulary Score: 50.05313372247758
Average Average Cohesion Score: 86.32605175767958
Average Conventions Score: 0.16196153846153846
Average Bleu Score: 0.09124955959884512
Average Sts Score: 0.6295220278776609
Average rouge Score: 0.4240295250053356


### BART

In [ ]:

data = pd.DataFrame({"context": [text_data]*len(questions), "questions": questions, "answers": answers, "improved_answers": improved_answers})

model_name = "facebook/bart-large"
bart_tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForConditionalGeneration.from_pretrained(model_name)

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length=128, max_target_length=50):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        # source_text = "question: " + item["question"] + " answer: " + item["original_answer"]
        source_text = f" questions: {item['questions']} answers: {item['answers']}"
        target_text = item["improved_answers"]
        source = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = source["input_ids"].squeeze()  # Remove the extra dimension
        attention_mask = source["attention_mask"].squeeze()  # Remove the extra dimension
        labels = target["input_ids"].squeeze()  # Remove the extra dimension
        decoder_attention_mask = target["attention_mask"].squeeze()  # Remove the extra dimension
        return {"input_ids": input_ids, "attention_mask": attention_mask,
                "labels": labels, "decoder_attention_mask": decoder_attention_mask}

dataset = CustomDataset(data, bart_tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart_model.to(device)
optimizer = torch.optim.AdamW(bart_model.parameters(), lr=5e-5)

for epoch in range(3):  # Number of epochs
    bart_model.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = bart_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")

sample_question = "What are the potential applications of AI in agriculture?"
sample_answer = "In agriculture, AI can be used for tasks such as crop monitoring, yield prediction, pest detection, and automated farming equipment control. It can analyze data from sensors, drones, and satellite imagery to provide insights for optimizing crop production and resource management."

generated_answer = bart_tokenizer.decode(bart_model.generate(bart_tokenizer(sample_question, sample_answer, return_tensors="pt")["input_ids"], max_length=50)[0], skip_special_tokens=True)

# Print the generated answer
print("Generated Answer:", generated_answer)
